# Evaluation Results

In [6]:
import os
from pathlib import Path
from semconstmining.config import Config
import warnings
warnings.filterwarnings('ignore')
res_file = "1688065124.331203_run.csv"
conf = Config(Path(os.getcwd()).parents[1].resolve(), "sap_sam_filtered_2500")

import pandas as pd
import matplotlib.pyplot as plt

run_results_df = pd.read_csv(conf.DATA_EVAL / res_file)
run_results_df

,config,log_id,tp,fp,fn,precision,recall,f1,support,run_time,...,Resource_f1,Resource_support,Object_precision,Object_recall,Object_f1,Object_support,Multi-object_precision,Multi-object_recall,Multi-object_f1,Multi-object_support
0,0,1cdc0bb467ec450c8cd349e2bdd80139,150,4653,11,0.031230,0.931677,0.060435,161,7.90,...,0.456621,100,1.000000,0.500000,0.666667,22,0.008057,1.0,0.015985,35
1,0,1d7e74365d8547499d95382130056464,7,0,15,1.000000,0.318182,0.482759,22,6.54,...,0.000000,0,1.000000,0.318182,0.482759,22,0.000000,0.0,0.000000,0
2,0,1e05b9525bbb4cc4a92e241e4aacf8f4,38,235,31,0.139194,0.550725,0.222222,69,5.78,...,0.000000,0,1.000000,0.500000,0.666667,62,0.000000,0.0,0.000000,0
3,0,410aec05d68144be943693a9ebb7d0b8,108,0,7,1.000000,0.939130,0.968610,115,6.48,...,1.000000,94,1.000000,0.500000,0.666667,14,0.000000,0.0,0.000000,0
4,0,644dc442735a470899ff8ee1f50ae7df,251,629,119,0.285227,0.678378,0.401600,370,6.28,...,1.000000,192,0.224138,0.179310,0.199234,145,0.007366,1.0,0.014625,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1289,1,e84a91be294a4d6db142ad069b7ad204,19,0,19,1.000000,0.500000,0.666667,38,5.53,...,0.000000,0,1.000000,0.500000,0.666667,38,0.000000,0.0,0.000000,0
1290,1,16b56d38130548148044e7069df7d398,17,387,85,0.042079,0.166667,0.067194,102,5.92,...,0.000000,0,0.042079,0.166667,0.067194,102,0.000000,0.0,0.000000,0
1291,1,16fc39198c21474ba8e9821d54f0b2f0,14,529,472,0.025783,0.028807,0.027211,486,5.68,...,0.000000,0,0.025783,0.028807,0.027211,486,0.000000,0.0,0.000000,0
1292,1,17060ea3275245ae98bdb2f01528fc2e,0,0,52,0.000000,0.000000,0.000000,52,5.54,...,0.000000,0,0.000000,0.000000,0.000000,52,0.000000,0.0,0.000000,0


In [11]:
import time

const_types = [conf.ACTIVITY, conf.RESOURCE, conf.OBJECT, conf.MULTI_OBJECT]
avg_frames = []
for config, group in run_results_df.groupby("config"):
    config_results = {}
    avg_precision = group.loc[group["support"] > 0, "precision"].mean()
    avg_recall = group.loc[group["support"] > 0, "recall"].mean()
    avg_f1 = group.loc[group["support"] > 0, "f1"].mean()
    avg_support = group.loc[group["support"] > 0, "support"].mean()
    total_support = group["support"].sum()
    avg_run_time = group["run_time"].mean()
    tp_avg = group["tp"].mean()
    fp_avg = group["fp"].mean()
    fn_avg = group["fn"].mean()
    tp_total = group["tp"].sum()
    fp_total = group["fp"].sum()
    fn_total = group["fn"].sum()
    config_results["config"] = config
    config_results["avg_precision"] = avg_precision
    config_results["avg_recall"] = avg_recall
    config_results["avg_f1"] = avg_f1
    config_results["avg_support"] = avg_support
    config_results["total_support"] = total_support
    config_results["avg_run_time"] = avg_run_time
    config_results["tp_avg"] = tp_avg
    config_results["fp_avg"] = fp_avg
    config_results["fn_avg"] = fn_avg
    config_results["tp_total"] = tp_total
    config_results["fp_total"] = fp_total
    config_results["fn_total"] = fn_total
    per_const_dict = {}
    for const_type in const_types:
        const_type_avg_precision = group.loc[group[f"{const_type}_support"] > 0, f"{const_type}_precision"].mean()
        const_type_avg_recall = group.loc[group[f"{const_type}_support"] > 0, f"{const_type}_recall"].mean()
        const_type_avg_f1 = group.loc[group[f"{const_type}_support"] > 0, f"{const_type}_f1"].mean()
        const_type_avg_support = group.loc[group[f"{const_type}_support"] > 0, f"{const_type}_support"].mean()
        const_type_total_support = group[f"{const_type}_support"].sum()
        # const_tp_avg = group[f"{const_type}_tp"].mean()
        # const_fp_avg = group[f"{const_type}_fp"].mean()
        # const_fn_avg = group[f"{const_type}_fn"].mean()
        # const_tp_total = group[f"{const_type}_tp"].sum()
        # const_fp_total = group[f"{const_type}_fp"].sum()
        # const_fn_total = group[f"{const_type}_fn"].sum()
        per_const_dict |= {
            f"{const_type}_precision": const_type_avg_precision,
            f"{const_type}_recall": const_type_avg_recall,
            f"{const_type}_f1": const_type_avg_f1,
            f"{const_type}_support": const_type_avg_support,
            f"{const_type}_total_support": const_type_total_support,
            # f"{const_type}_tp": const_tp_avg,
            # f"{const_type}_fp": const_fp_avg,
            # f"{const_type}_fn": const_fn_avg,
            # f"{const_type}_total_tp": const_tp_total,
            # f"{const_type}_total_fp": const_fp_total,
            # f"{const_type}_total_fn": const_fn_total,
        }
    config_results|= per_const_dict
    avg_frames.append(config_results)
avg_results_df = pd.DataFrame.from_records(avg_frames)
avg_results_df.to_csv(conf.DATA_EVAL / f"average_{time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime())}.csv",
                        index=False)
avg_results_df

,config,avg_precision,avg_recall,avg_f1,avg_support,total_support,avg_run_time,tp_avg,fp_avg,fn_avg,...,Object_precision,Object_recall,Object_f1,Object_support,Object_total_support,Multi-object_precision,Multi-object_recall,Multi-object_f1,Multi-object_support,Multi-object_total_support
0,0,0.336119,0.493530,0.260766,389.613115,237664,6.230974,99.340031,991.799073,267.992272,...,0.604475,0.313698,0.350251,336.647766,195929,0.09296,1.0,0.130409,56.220238,9445
1,1,0.600893,0.273166,0.315065,356.892675,209496,5.841561,49.435858,61.948995,274.360124,...,0.603265,0.230069,0.287919,336.647766,195929,NaN,NaN,NaN,NaN,0


In [12]:
import os
from pathlib import Path
from semconstmining.config import Config
import pandas as pd
import time
conf = Config(Path(os.getcwd()).parents[1].resolve(), "sap_sam_filtered_2500")
log_dir = conf.DATA_EVAL / (conf.MODEL_COLLECTION + "_noisy.pkl")
logs = pd.read_pickle(log_dir)

cnt = 0
for idx, row in logs.iterrows():
    log = row[conf.LOG]
    noisy_log =  row[conf.NOISY_LOG]
    if len(row[conf.LOG]._list) <=1:
        print(row[conf.MODEL_ID])
        cnt += 1

print(cnt)

1be852b48fb24bfc8aab46caf7cf0928
1c0fdaf892db48759871b78bf54bba6b
1c2f94c5662b4537ad3e716a80ac89ca
1c45c69b93b347e29ff3dea0a731e1c1
1da20cfa717944168f0e47e975d87b76
1de94027e693495fb89d28cc22af81c8
1e00258995924821be2f772d889290d6
410aec05d68144be943693a9ebb7d0b8
411d652b049547dfa0a15dee30f0e440
6483491dd000442894c4a79d784590b4
64dfe71bbd8446c19fb48a468781fc01
6519a97275ed4d078e3a1091bce4fd15
65a18a6a4df04dc4bceac71e8563515e
65b1af0fb6c0464a82e48513afc1460c
660d830ba5ce4dfe9cc85e102fadfbf6
662c096587b14aef9990ca4b26739965
39d34b6f9d67499e9f5a72f52d53f412
3a1400c927a146479d5f90a5e8b0b4a2
3a184d0a00c448c6bd17636b7ce6dc71
3b56490e9fb6417b8a6d8c191e97ee74
3b78d4323e754b50b5d007168e0a5b66
3be368b0244a49f09126a57d618b2804
18cd0986b56849949617db8ea30db7b3
191131cd6be54ff2b14149f0b8fce4dd
19117310241d45b3bd465c8947e060a4
0a514af86aa5468f8955efc22e63c4da
0ae2d60656874fb2b5673507a81a1d38
0b2009450ac74f88a52ed06c9278cc00
0b375496f4bc4b5db10a3294ad018a94
0b5fe3e2c0d94df8b4e2f310d0938c03
0b681b2413